In [1]:
#pip install stockstats

In [1]:
import os
import pandas as pd
import csv
import numpy as np
from stockstats import wrap
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

In [2]:
# get files
def files(dir):
    names = []
    for root, dirs, files in os.walk(dir):  
        for file in files:  
            if os.path.splitext(file)[1] == '.csv':  # csv files
                names.append(file)
    return names

In [3]:
names = files(os.getcwd() + "/data")

In [4]:
aug28_actual_price=[]
aug28_pred_price=[]

for file in names:
    df = pd.read_csv("data/"+ file,index_col='Date')
    df.index = pd.to_datetime(df.index) 
    ts_open=df['Open'] 
    ts_close=df['Close'] 
   
        # Test for stationarity
    def test_stationarity(timeseries):
        # Determing rolling statistics
        rolmean = timeseries.rolling(12).mean()
        rolstd = timeseries.rolling(12).std()
        adft = adfuller(timeseries,autolag='AIC')
        output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
        for key,values in adft[4].items():
            output['critical value (%s)'%key] =  values
    test_stationarity(ts_open)
    test_stationarity(ts_close)
    
    #if not stationary then eliminate trend
    #Eliminate trend
    moving_avg = ts_open.rolling(12).mean()
    std_dev = ts_open.rolling(12).std()
    
    moving_avg = ts_close.rolling(12).mean()
    std_dev = ts_close.rolling(12).std()
    
     #split data into train and test set
    train_data, test_data = ts_close[:pd.to_datetime('2021-08-27')], ts_close[pd.to_datetime('2021-08-27'):]

    model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
    
    model = sm.tsa.arima.ARIMA(train_data, order=model_autoARIMA.order)
    fitted = model.fit()
    
    close_pred = fitted.forecast(test_data.size, alpha=0.05)
    # close_pred_series = pd.Series(close_pred, index=test_data.index)
    
    open_actual = pd.Series(ts_open, index=ts_open.index)
     
    aug28_actual_price.append(open_actual.get(key = '2021-08-28'))
    aug28_pred_price.append(close_pred.iloc[0])
    
print(aug28_actual_price)
print(aug28_pred_price)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=193.262, Time=0.44 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=193.969, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=194.047, Time=0.18 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=191.287, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=195.875, Time=0.33 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.161 seconds


C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3068.142, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3066.963, Time=0.30 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3067.369, Time=0.21 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3066.789, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3058.023, Time=0.50 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3058.762, Time=0.76 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3058.320, Time=0.88 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3068.011, Time=0.27 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3066.905, Time=0.40 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3042.041, Time=1.72 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=3044.185, Time=1.83 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=3044.035, Time=1.44 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3053.278, Time=0.75 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3057.751, Time=0.78 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept

C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=1669.960, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.34 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=680.001, Time=0.23 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-2033.140, Time=0.50 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-2036.939, Time=1.10 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=inf, Time=0.60 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=-2061.042, Time=1.60 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=inf, Time=1.20 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=-2054.366, Time=2.26 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-2059.224, Time=2.52 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=-2060.563, Time=2.87 sec

Best model:  ARIMA(3,0,1)(0,0,0)[0]          
Total fit time: 13.341 seconds


...

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=11506.106, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=11499.768, Time=0.43 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=11498.417, Time=0.53 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=11504.143, Time=0.08 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=11497.775, Time=1.08 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=11491.530, Time=1.33 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=11493.412, Time=0.31 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=11492.020, Time=2.02 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=11435.377, Time=5.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=11493.432, Time=1.24 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=11432.496, Time=5.43 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=11437.335, Time=3.88 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=11432.681, Time=4.72 sec
 ARIMA(3,1,2)(0,0,0)[0]             : AIC=11430.519, Time=2.39 sec
 ARIMA(2,1,2)(0,0,0

C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-9625.335, Time=0.34 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-9666.291, Time=1.30 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-9659.943, Time=1.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-9627.324, Time=0.25 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-9670.648, Time=1.23 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-9670.335, Time=2.39 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-9676.845, Time=2.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-9675.265, Time=2.09 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-9667.739, Time=4.67 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-9676.966, Time=2.99 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-9681.094, Time=3.05 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-9667.351, Time=3.03 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-9677.140, Time=2.10 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-9665.320, Time=1.41 sec
 ARIMA(2,1,3)(0,0,0

C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\86173\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


[3.2150890827178955, 372.9949340820313, 1.0661139488220217, 38.53634643554688, 50.11, 73.68887329101562, 0.8451420068740845, 0.9999639987945556, 48269.0, 635.97, 69.49895477294922, 161.7786102294922, 49062.8, 0.0042869998142123, 494.6706237792969, 2.946, 4.163629055023193, 5.866648197174072, 26.04, 0.37813600897789, 7.002458095550537, 20.4457, 0.1568839997053146, 2.1836259365081787, 0.9997, 242.52386474609372, 0.930428981781006, 179.07078552246094, 0.289669, 9.40000027185306e-05, 141.037841796875, 2.0037429332733154, 5.090341091156006, 64.0815, 3273.78, 0.4923639893531799, 0.9929310083389282, 75.22450256347656, 22.166019439697266, 49.4881, 0.0313020013272762, 299.32391357421875, 0.2540949881076813, 22.797870635986328, 0.0140479998663067, 14.368988990783691, 62.76066589355469, 1.0427169799804688, 8.160792350769043, 1.7443510293960571, 13.344459533691406, 306.1820678710937, 6.071691036224365, 176.13, 0.4978469908237457, 3752.0244140625, 2.963104009628296, 315.037, 5.315, 0.19987599551677

In [5]:
out = pd.DataFrame(list(zip(names, aug28_actual_price, aug28_pred_price)),
                    columns=['crypto', 'actual price 8.28','pred price 8.28'])
                    
out['actual price 8.28'] = out['actual price 8.28'].astype('float64')
out['pred price 8.28'] = out['pred price 8.28'].astype('float64')
out['crypto'] = out['crypto'].str[:-4]
out.to_csv('output_arima_final.csv')
out.sort_values(by=['actual price 8.28'], ascending=False)
out.sort_values(by=['pred price 8.28'], ascending=False)

,crypto,actual price 8.28,pred price 8.28
12,bitcoin,49062.800000,48771.066642
8,Bitcoin BEP2,48269.000000,48045.123553
55,Maker,3752.024414,3698.349922
34,ethereum,3273.780000,3217.317980
66,PAX Gold,1822.720825,1817.224519
...,...,...,...
40,Gala,0.031302,0.031270
44,Holo,0.014048,0.014538
13,BitTorrent (New),0.004287,0.004187
29,eCash,0.000094,0.000094
